모듈 등록(import)

In [19]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder as le
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier

매개변수 및 데이터 입력
매개변수 사상

In [20]:
ROOT_FILENAME = "../input/"
#ROOT_FILENAME = ""
TRAIN_FILENAME = 'train.csv'
TEST_FILENAME = 'test.csv'
RESULT_FILENAME = 'res2.cv'

train.csv와 test.csv 투입

In [21]:
train = pd.read_csv(ROOT_FILENAME+TRAIN_FILENAME, parse_dates=['Dates'], index_col=False)
print(train.shape)
train.head(3)

(878049, 9)


,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414


In [22]:
test = pd.read_csv(ROOT_FILENAME+TEST_FILENAME, parse_dates=['Dates'], index_col=False)
print(test.shape)
test.head(3)

(884262, 7)


,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212


데이터 마이닝(탐구)
: 우리는 시험 중에 이용할 수 있는 밭을 확인할 것이다. (게임이 캐글에게 전송해야 할 시험 게임) 그러므로 모든 밭은 설명과 품목의 허용에 대하여 조사될 것이다.

In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
Dates         878049 non-null datetime64[ns]
Category      878049 non-null object
Descript      878049 non-null object
DayOfWeek     878049 non-null object
PdDistrict    878049 non-null object
Resolution    878049 non-null object
Address       878049 non-null object
X             878049 non-null float64
Y             878049 non-null float64
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 60.3+ MB


데이터 전송

In [24]:
categories = {c:i for i,c in enumerate(train['Category'].unique())}
cat_rev = {i:c for i,c in enumerate(train['Category'].unique())}

In [25]:
districts = {c:i for i,c in enumerate(train['PdDistrict'].unique())}

In [26]:
weekdays = {'Monday':0., 'Tuesday':1., 'Wednesday':2., 'Thursday': 3., 'Friday':4., 'Saturday':5., 'Sunday':6.}
weekdays2 = {'Monday':0., 'Tuesday':0., 'Wednesday':0., 'Thursday': 0., 'Friday':0., 'Saturday':1., 'Sunday':1}

In [27]:
def getHourZn(hour):
    if(hour >= 2 and hour < 8): return 1;
    if(hour >= 8 and hour < 12): return 2;
    if(hour >= 12 and hour < 14): return 3;
    if(hour >= 14 and hour < 18): return 4;
    if(hour >= 18 and hour < 22): return 5;
    if(hour < 2 or hour >= 22): return 6;


In [28]:
def define_address(addr):
    addr_type = 0.
    # Address types:
    #  Intersection: 1
    #  Residence: 0categories
    if '/' in addr and 'of' not in addr:
        addr_type = 1.
    else:
        add_type = 0.
    return addr_type

In [29]:
def feature_engineering(data):
    data['Day'] = data['Dates'].dt.day
    data['Month'] = data['Dates'].dt.month
    data['Year'] = data['Dates'].dt.year-2003
    data['Hour'] = data['Dates'].dt.hour
    data['Minute'] = data['Dates'].dt.minute
    data['Day_Num'] = [float(weekdays[w]) for w in data.DayOfWeek]
    data['WeekOfYear'] = data['Dates'].dt.weekofyear
    data['District_Num'] = [float(districts[t]) for t in data.PdDistrict]
    data['HourZn'] = preprocessing.scale(list(map(getHourZn, data['Dates'].dt.hour)))
    data['isWeekday'] = [float(weekdays2[w]) for w in data.DayOfWeek]
    data['X'] = preprocessing.scale(list(map(lambda x: x+122.4194, data.X)))
    data['Y'] = preprocessing.scale(list(map(lambda x: x-37.7749, data.Y)))
    data['Address_Type'] = list(map(define_address, data.Address))
#   data['HourZn'] = getHourZn(data['Dates'].dt.hour);
    return data

In [30]:
X_loc = ['X', 'Y', 'District_Num', 'Address_Type']
X_time = ['Minute', 'Hour']
X_date = ['Year','Month', 'Day', 'Day_Num', 'HourZn']
X_all = X_loc + X_time + X_date

train = feature_engineering(train)
train['Category_Num'] = [float(categories[t]) for t in train.Category]

X_train, X_test, y_train, y_test = train_test_split(train[X_all], train['Category_Num'], test_size = 0.2, random_state = 0)

test = feature_engineering(test)

랜섬 포레슈트
점수가 0인 점을 고려해볼 때, 정밀도, 축구 경기, 그리고 F1을 살펴볼 것이다.

네 가지 선택이 있습니다.
 micro : 전체 참의 양, 거짓의 음수와 거짓의 양(false positive)을 세어 전체적으로 측정지표를 계산한다.
 매크로: 각 레이블에 대한 측정지표를 계산하고 가중치가 없는 평균을 구한다. 이것은 라벨 불균형을 고려하지 않는다.
 가중 : 각 레이블에 대한 메트릭을 계산하고, 각 레이블에 대한 실제 인스턴스 수(각 레이블에 대한 실제 인스턴스 수)를 기준으로 가중치를 부여한 평균을 구하십시오. 이것은 라벨 불균형을 설명하기 위해 '매크로'를 바꾼다. 그것은 정밀도와 리콜 사이에 있지 않은 F 점수를 초래할 수 있다.

In [31]:
#clf = RandomForestClassifier(max_features="log2", max_depth=11, n_estimators=24,
#                             min_samples_split=1000, oob_score=True).fit(X_train,y_train)
#y = clf.predict(X_test)

clf = RandomForestClassifier(max_features="log2", max_depth=14, n_estimators=25,
                             min_samples_split=300, oob_score=False).fit(X_train,y_train)
y = clf.predict(X_test)


recall_score(y,y_test, average='micro')
# recall_score_micro : 0.29731792039177724
recall_score(y,y_test, average='macro')
# recall_score_macro : 0.09619918771835194
recall_score(y,y_test, average='weighted')
# recall_score_weighted : 0.29731792039177724

precision_score(y,y_test, average='micro')
# precision_score_micro : 0.29731792039177724
precision_score(y,y_test, average='macro')
# precision_score_macro : 0.07442702033391868
precision_score(y,y_test, average='weighted')
# precision_score_weighted : 0.567191672388691

accuracy_score(y,y_test)
# accuracy_score : 0.29731792039177724


f1_score(y,y_test, average='micro')
# F1_micro : 0.29731792039177724
f1_score(y,y_test, average='macro')
# F1_macro : 0.0683830234838435
f1_score(y,y_test, average='weighted')
# F1_weighted : 0.3704176711997292

#clf = RandomForestClassifier(max_features="log2", max_depth=14, n_estimators=25,
#                             min_samples_split=300, oob_score=False).fit(train[X_all],train['Category_Num'])
#y = clf.predict_proba(test[X_all])
#submission = pd.DataFrame({cat_rev[p] : [y[i][p] for i in range(len(y))] for p in range(len(y[0]))})

#submission['Id'] = [i for i in range(len(submission))]
#submission.to_csv('result_RN.csv', index=False)
# # 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.3698860791090158

In [34]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y = knn.predict(X_test)


recall_score(y,y_test, average='micro')
# recall_score_micro : 0.1996640282444052
recall_score(y,y_test, average='macro')
# recall_score_macro : 0.07300143475941939
recall_score(y,y_test, average='weighted')
# recall_score_weighted : 0.1996640282444052

precision_score(y,y_test, average='micro')
# precision_score_micro : 0.1996640282444052
precision_score(y,y_test, average='macro')
# precision_score_macro : 0.052448254827175196
precision_score(y,y_test, average='weighted')
# precision_score_weighted : 0.2742525234494359

accuracy_score(y,y_test)
# accuracy_score : 0.1996640282444052


f1_score(y,y_test, average='micro')
# F1_micro : 0.19966402824440524
f1_score(y,y_test, average='macro')
# F1_macro : 0.05236657213517695
f1_score(y,y_test, average='weighted')
# F1_weighted : 0.22499597882768185


#y = knn.predict_proba(X_test)

#submission = pd.DataFrame({cat_rev[p] : [y[i][p] for i in range(len(y))] for p in range(len(y[0]))})
#submission['Id'] = [i for i in range(len(submission))]

#submission.to_csv("submission_knn.csv", index=False) 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.22499597882768185

In [39]:
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression()
lgr.fit(X_train,y_train)
y = lgr.predict(X_test)


recall_score(y,y_test, average='micro')
recall_score(y,y_test, average='macro')
recall_score(y,y_test, average='weighted')

precision_score(y,y_test, average='micro')
precision_score(y,y_test, average='macro')
precision_score(y,y_test, average='weighted')

accuracy_score(y,y_test)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.21987358350891179

In [40]:

f1_score(y,y_test, average='micro')
# F1_micro : 0.21990775012812483
f1_score(y,y_test, average='macro')
# F1_macro : 0.0207335350195717
f1_score(y,y_test, average='weighted')
# F1_weighted : 0.3166197259646911

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.3165873207052942

In [41]:
#seed = 37

#model = xgb.XGBClassifier(objective='multi:softprob',
#                          n_estimators=45,
#                          learning_rate=1.0,
#                          max_depth=1,
#                          max_delta_step=1,
#                          nthread=-1,
#                          seed=seed)
#model.fit(X_train, y_train)
#predictions = model.predict_proba(X_test)


#y = model.predict_proba(test[X_all])

#submission = pd.DataFrame({cat_rev[p] : [y[i][p] for i in range(len(y))] for p in range(len(y[0]))})
#submission['Id'] = [i for i in range(len(submission))]
#submission.to_csv('result_logistic.csv', index=False)